<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Better and consistent library that makes the same function as the code above



In [1]:
! pip install pyspark py4j
! pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**Importing modules and pandas library that will be used later.**

In [2]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
import findspark

findspark.init()

In [3]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

23/07/03 15:19:27 WARN Utils: Your hostname, Linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/03 15:19:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/03 15:19:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/03 15:19:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Gooogle Colab Version

In [164]:
""" 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 
"""

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [4]:
fille = r"../RAW_DATA/Parquet/BrStocksFormated.parquet"

stocks = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [5]:
fille = r"../RAW_DATA/Parquet/Ebit.parquet"

ebit = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [6]:
fille = r"../RAW_DATA/Parquet/Stat.parquet"

stat = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [7]:
fille = r"../RAW_DATA/Parquet/Price.parquet"

price = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

In [8]:
fille =r'../RAW_DATA/Parquet/Balance.parquet'

balance = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [9]:
# Stocks table
stocks.createOrReplaceTempView("General")
# Ebit table
ebit.createOrReplaceTempView("Ebit")
# Stat table
stat.createOrReplaceTempView("Stat")
# Price table
price.createOrReplaceTempView("Price")
# Balance table
balance.createOrReplaceTempView("Balance")

23/07/03 15:20:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

In [10]:
# Already called at Calc Table
Gen = spark.sql("""
SELECT * FROM General
WHERE Moeda != 'USD' AND EBITDA != 0 AND ROA != 0 AND ROE != 0;
""").createOrReplaceTempView("Gen")

In [172]:
#Gen.show()

In [11]:
# Already called at RSCH table
Ebit = spark.sql("""
SELECT * FROM Ebit
where Divida_Liquida != 0;
""").createOrReplaceTempView("Debt")

In [174]:
#Ebit.show()

In [12]:
# Already called at RSCH table
Price = spark.sql("""
SELECT * FROM Price
WHERE valor_de_mercado != 0;
""")#.createOrReplaceTempView("Prices")

In [13]:
Price.show()

+--------+----------------+-----------------+-----------------+
|  Ticker|valor_de_mercado|volume_de_mercado|__index_level_0__|
+--------+----------------+-----------------+-----------------+
|ZAMP3.SA|      1290876288|                0|                0|
|AERI3.SA|       964650304|                0|                3|
|DOTZ3.SA|       113162640|                0|                5|
|MLAS3.SA|      1763443968|                0|                8|
|NINJ3.SA|       134109408|                0|               11|
|MODL3.SA|      1179109888|                0|               14|
|VITT3.SA|      1594519040|                0|               15|
|KRSA3.SA|       874195328|                0|               16|
|ALLD3.SA|       470764032|                0|               23|
|ATMP3.SA|        27776526|                0|               25|
|JSLG3.SA|      1982957056|                0|               27|
|ELMD3.SA|      1802294016|                0|               29|
|OPCT3.SA|       617591488|             

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [14]:
Stat = spark.sql("""
SELECT * FROM Stat
WHERE Beta != 0.5;
""")#.createOrReplaceTempView("Stat")

In [178]:
#Stat.show()

In [15]:
CALCULATE = spark.sql("""
SELECT * FROM Gen
WHERE Margem_Bruta > 0.0 AND Margem_EBITIDA > 0.0 AND 
Margem_Operacional > 0.0 AND Margem_liquida > 0.0 AND 
ROE > 0 AND Crescimento_de_Receita_3T > 0;
""")#.createOrReplaceTempView("Calc")

In [180]:
#CALCULATE.show()

In [16]:
RSCH = spark.sql("""
SELECT p.Ticker, 
       p.valor_de_mercado, 
       p.volume_de_mercado, 
       e.EBIT, 
       e.Divida_Liquida 
FROM Price as p, Ebit as e
WHERE (p.valor_de_mercado + e.Divida_Liquida)/e.EBIT > 0.0 AND 
p.Ticker == e.Ticker; 
""")#.createOrReplaceTempView("RSCH")

In [53]:
Balance = spark.sql("""
SELECT  Ticker,
        Moeda,
        Receita_a_Pagar,
        Receitas_a_Receber,
        Bens_de_Capital
FROM Balance
""")

In [54]:
Balance.show()

+---------+-----+---------------+------------------+---------------+
|   Ticker|Moeda|Receita_a_Pagar|Receitas_a_Receber|Bens_de_Capital|
+---------+-----+---------------+------------------+---------------+
|A2MC34.SA|  USD|        5.433E8|           1.603E8|      1000000.0|
|A2MC34.SA|  USD|        2.988E8|            2.31E7|      2300000.0|
|A2MC34.SA|  USD|        3.771E8|            8.54E7|      5100000.0|
| AALR3.SA|  BRL|        5.667E7|          8.8789E7|      6.12412E8|
| AALR3.SA|  BRL|       9.4887E7|          7.9679E7|      6.12412E8|
| AALR3.SA|  BRL|      1.17548E8|         1.04502E8|      6.12412E8|
|AAPL34.SA|  USD|      4.6236E10|         2.2926E10|      4.5174E10|
|AAPL34.SA|  USD|      4.2296E10|          1.612E10|      5.0779E10|
|AAPL34.SA|  USD|      5.4763E10|         2.6278E10|      5.7365E10|
|AAPL34.SA|  USD|      6.4115E10|         2.8184E10|      6.4849E10|
|ABTT34.SA|  USD|        3.252E9|           5.425E9|      2.3853E10|
|ABTT34.SA|  USD|        3.946E9| 

In [59]:
calendario = spark.sql("""
SELECT 
    CAST(Data_de_Reporte AS date) AS Data_Reporte
FROM Balance 
""")

In [60]:
calendario.show()

+------------+
|Data_Reporte|
+------------+
|  2019-12-30|
|  2020-12-30|
|  2021-12-30|
|  2019-12-30|
|  2020-12-30|
|  2021-12-30|
|  2019-09-29|
|  2020-09-29|
|  2021-09-29|
|  2022-09-29|
|  2019-12-30|
|  2020-12-30|
|  2021-12-30|
|  2022-12-30|
|  2019-12-30|
|  2020-12-30|
|  2021-12-30|
|  2022-12-30|
|  2019-12-30|
|  2020-12-30|
+------------+
only showing top 20 rows



ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

Pandas setting DataFrame to pandas format:

In [61]:
df_stat = Stat.toPandas()

df_calc = CALCULATE.toPandas()

df_rsc = RSCH.toPandas()

df_balance = Balance.toPandas()

df_calendario = calendario.toPandas()

In [62]:
df_balance

,Ticker,Moeda,Receita_a_Pagar,Receitas_a_Receber,Bens_de_Capital
0,A2MC34.SA,USD,543300000.0,160300000.0,1.000000e+06
1,A2MC34.SA,USD,298800000.0,23100000.0,2.300000e+06
2,A2MC34.SA,USD,377100000.0,85400000.0,5.100000e+06
3,AALR3.SA,BRL,56670000.0,88789000.0,6.124120e+08
4,AALR3.SA,BRL,94887000.0,79679000.0,6.124120e+08
...,...,...,...,...,...
1025,YDUQ3.SA,BRL,251229000.0,890151000.0,1.139887e+09
1026,YDUQ3.SA,BRL,183529000.0,957746000.0,1.139887e+09
1027,ZAMP3.SA,BRL,286432000.0,60361000.0,9.507680e+08
1028,ZAMP3.SA,BRL,217616000.0,105011000.0,1.461068e+09


Excel format tables

In [134]:
# df_stat.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Excel/Stat.xlsx",
#                  sheet_name="stat")

# df_calc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Calcl.xlsx",
#                  sheet_name="calc")

# df_rsc.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/RSCH.xlsx",
#                 sheet_name="research")

# df_balance.to_excel(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Balance.xlsx",
#                 sheet_name="balance")

Parquet Format tables

In [63]:
df_balance.to_parquet(r"../Final/Parquet/Balance.parquet")

df_stat.to_parquet(r"../Final/Parquet/Calcl.parquet")

df_rsc.to_parquet(r"../Final/Parquet/RSCH.parquet")

df_calc.to_parquet(r"../Final/Parquet/Calcl.parquet")

df_calendario.to_parquet(r"../Final/Parquet/Calendario.parquet")